In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Meteorological data\ASOS\CLL_1992-2022.csv')
data

,station,valid,tmpc
0,CLL,1992-01-01 00:00,8.89
1,CLL,1992-01-01 01:00,8.89
2,CLL,1992-01-01 02:00,8.33
3,CLL,1992-01-01 03:00,7.78
4,CLL,1992-01-01 04:00,7.78
...,...,...,...
324417,CLL,2022-12-31 19:53,18.33
324418,CLL,2022-12-31 20:53,18.33
324419,CLL,2022-12-31 21:53,17.22
324420,CLL,2022-12-31 22:53,15.00


In [3]:
data['valid'] = pd.to_datetime(data['valid'])

data['Year'] = data['valid'].dt.year
data['Month'] = data['valid'].dt.month
data['Day'] = data['valid'].dt.day
data['Time'] = data['valid'].dt.time

data

,station,valid,tmpc,Year,Month,Day,Time
0,CLL,1992-01-01 00:00:00,8.89,1992,1,1,00:00:00
1,CLL,1992-01-01 01:00:00,8.89,1992,1,1,01:00:00
2,CLL,1992-01-01 02:00:00,8.33,1992,1,1,02:00:00
3,CLL,1992-01-01 03:00:00,7.78,1992,1,1,03:00:00
4,CLL,1992-01-01 04:00:00,7.78,1992,1,1,04:00:00
...,...,...,...,...,...,...,...
324417,CLL,2022-12-31 19:53:00,18.33,2022,12,31,19:53:00
324418,CLL,2022-12-31 20:53:00,18.33,2022,12,31,20:53:00
324419,CLL,2022-12-31 21:53:00,17.22,2022,12,31,21:53:00
324420,CLL,2022-12-31 22:53:00,15.00,2022,12,31,22:53:00


In [4]:
data['tmpc'] = pd.to_numeric(data['tmpc'], errors='coerce')
data['tmpc'] = data['tmpc'].interpolate(method='linear')

In [5]:
daily_max_temp = data.groupby(['Year', 'Month', 'Day'])['tmpc'].max().reset_index()
daily_max_temp.rename(columns={'tmpc': 'DailyMaxTemp'}, inplace=True)
daily_max_temp

,Year,Month,Day,DailyMaxTemp
0,1992,1,1,18.89
1,1992,1,2,16.67
2,1992,1,3,20.00
3,1992,1,4,17.22
4,1992,1,5,12.78
...,...,...,...,...
11318,2022,12,27,11.11
11319,2022,12,28,24.44
11320,2022,12,29,23.89
11321,2022,12,30,18.33


In [6]:
daily_max_temp1 = daily_max_temp[daily_max_temp['Year'] == 2022]
daily_max_temp1

,Year,Month,Day,DailyMaxTemp
10958,2022,1,1,27.78
10959,2022,1,2,6.67
10960,2022,1,3,12.22
10961,2022,1,4,18.89
10962,2022,1,5,23.33
...,...,...,...,...
11318,2022,12,27,11.11
11319,2022,12,28,24.44
11320,2022,12,29,23.89
11321,2022,12,30,18.33


In [7]:
threshold = daily_max_temp['DailyMaxTemp'].quantile(0.98)
threshold

38.33

In [8]:
daily_max_temp['Exceeds_threshold'] = daily_max_temp['DailyMaxTemp'] > threshold
daily_max_temp

,Year,Month,Day,DailyMaxTemp,Exceeds_threshold
0,1992,1,1,18.89,False
1,1992,1,2,16.67,False
2,1992,1,3,20.00,False
3,1992,1,4,17.22,False
4,1992,1,5,12.78,False
...,...,...,...,...,...
11318,2022,12,27,11.11,False
11319,2022,12,28,24.44,False
11320,2022,12,29,23.89,False
11321,2022,12,30,18.33,False


In [9]:
data_2022 = daily_max_temp[daily_max_temp['Year'] == 2022]
data_2022

,Year,Month,Day,DailyMaxTemp,Exceeds_threshold
10958,2022,1,1,27.78,False
10959,2022,1,2,6.67,False
10960,2022,1,3,12.22,False
10961,2022,1,4,18.89,False
10962,2022,1,5,23.33,False
...,...,...,...,...,...
11318,2022,12,27,11.11,False
11319,2022,12,28,24.44,False
11320,2022,12,29,23.89,False
11321,2022,12,30,18.33,False


In [11]:
data_2022['Exceeds_threshold_Shifted_1'] = data_2022['Exceeds_threshold'].shift(1, fill_value=False)
data_2022['Exceeds_threshold_Shifted_2'] = data_2022['Exceeds_threshold'].shift(2, fill_value=False)
data_2022['Exceeds_threshold_Shifted_-1'] = data_2022['Exceeds_threshold'].shift(-1, fill_value=False)
data_2022['Exceeds_threshold_Shifted_-2'] = data_2022['Exceeds_threshold'].shift(-2, fill_value=False)

consecutive_days = data_2022[
    (data_2022['Exceeds_threshold'] & data_2022['Exceeds_threshold_Shifted_1'] & data_2022['Exceeds_threshold_Shifted_2']) |
    (data_2022['Exceeds_threshold'] & data_2022['Exceeds_threshold_Shifted_-1'] & data_2022['Exceeds_threshold_Shifted_-2']) |
    (data_2022['Exceeds_threshold'] & data_2022['Exceeds_threshold_Shifted_1'] & data_2022['Exceeds_threshold_Shifted_-1'])
]

consecutive_days = consecutive_days.drop(columns=['Exceeds_threshold_Shifted_1', 'Exceeds_threshold_Shifted_2', 'Exceeds_threshold_Shifted_-1', 'Exceeds_threshold_Shifted_-2'])
consecutive_days

C:\Users\gongw\AppData\Local\Temp\ipykernel_40392\2152932848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2022['Exceeds_threshold_Shifted_1'] = data_2022['Exceeds_threshold'].shift(1, fill_value=False)
C:\Users\gongw\AppData\Local\Temp\ipykernel_40392\2152932848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2022['Exceeds_threshold_Shifted_2'] = data_2022['Exceeds_threshold'].shift(2, fill_value=False)
C:\Users\gongw\AppData\Local\Temp\ipykernel_40392\2152932848.py:4: SettingWithCopyWa

,Year,Month,Day,DailyMaxTemp,Exceeds_threshold
11144,2022,7,6,38.89,True
11145,2022,7,7,38.89,True
11146,2022,7,8,39.44,True
11147,2022,7,9,41.11,True
11148,2022,7,10,43.89,True
11149,2022,7,11,40.00,True
11150,2022,7,12,41.67,True
11151,2022,7,13,40.56,True
11155,2022,7,17,39.44,True
11156,2022,7,18,39.44,True
